# Reto 04-A - Generación Aumentada con Recuperación (RAG) para Datos Estructurados

## Introducción

En este notebook, exploraremos la aplicación práctica de RAG con un tipo de datos más manejable, es decir, datos estructurados como datos relacionales o datos de texto almacenados en archivos CSV. El objetivo principal es introducir un caso de uso específico que demuestre la utilización de Azure AI Search para extraer documentos relevantes y el poder de ChatGPT para abordar las partes relevantes del documento, proporcionando resúmenes concisos basados en los prompts del usuario. Tiene como objetivo mostrar cómo se pueden adaptar las capacidades de ChatGPT de Azure OpenAI a tus necesidades de resumen, al mismo tiempo que te guía a través de la configuración y evaluación de los resultados del resumen. Este método se puede personalizar para adaptarse a varios casos de uso de resumen y aplicarse a diversos conjuntos de datos.

Tus objetivos para este desafío son leer este notebook, ejecutar cada bloque de código, observar los resultados y luego poder responder las preguntas planteadas en la guía del estudiante.

## Caso de Uso

Este caso de uso consta de tres secciones:
- Búsqueda de Documentos: El proceso de extraer documentos relevantes en función de la consulta de un corpus de documentos.
- Búsqueda de Zona de Documentos: El proceso de encontrar la parte relevante del documento extraído de la búsqueda de documentos.
- Tareas de IA posteriores, como Responder a Preguntas (también conocidas como Resumen de Texto): el resumen de texto es el proceso de crear resúmenes a partir de grandes volúmenes de datos manteniendo elementos informativos significativos y valor de contenido.

Este caso de uso puede ser útil para ayudar a expertos en la materia a encontrar información relevante en un gran corpus de documentos.

**Ejemplo:** En el proceso de descubrimiento de medicamentos, los científicos de la industria farmacéutica leen un corpus de documentos para encontrar información específica relacionada con conceptos, resultados de experimentos, etc. Este caso de uso les permite hacer preguntas al corpus de documentos y la solución devolverá la respuesta sucinta. En consecuencia, se acelera el proceso de descubrimiento de medicamentos.
 
Beneficios de la solución:
1. Acorta el tiempo de lectura.
2. Mejora la efectividad de la búsqueda de información.
3. Elimina el sesgo de las técnicas de resumen humanas.
4. Aumenta la capacidad para que los humanos se enfoquen en análisis más profundos.

La necesidad de resumen de documentos puede aplicarse a cualquier tema (legal, financiero, periodístico, médico, académico, etc.) que requiera un resumen de documentos largos. El tema en el que se centra este notebook es periodístico: recorreremos artículos de noticias.

### Conjunto de Datos de CNN y Daily Mail
Para este tutorial, utilizaremos el conjunto de datos de CNN/Daily Mail. Se trata de un conjunto de datos comúnmente utilizado para tareas de resumen de texto y respuesta a preguntas. Se generaron resúmenes abstractivos humanos a partir de historias de noticias en los sitios web de CNN y Daily Mail.

### Descripción de los Datos
El esquema relevante para nuestro trabajo de hoy consiste en:

- `id`: una cadena que contiene el hash SHA1 con formato hexadecimal de la URL de donde se recuperó la noticia.
- `article`: una cadena de caracteres que contiene el cuerpo del artículo de noticias.
- `highlights`: una cadena de caracteres que contiene lo más destacado del artículo tal como lo escribió el autor del artículo

In [1]:
# Import Azure Cognitive Search, OpenAI, and other python modules

import os, json, requests, sys, re
import requests
from pprint import pprint
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)


import openai
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# This is secure and recommended way to load OpenAI resource credentials and deployment names

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model=os.environ['EMBEDDING_MODEL_NAME']

**NOTA:** La ruta en la celda de código a continuación hace referencia al archivo `cnn_dailymail.csv` en la carpeta `/data/structured/`.

In [3]:
# read the CNN dailymail dataset in pandas dataframe
df = pd.read_csv('../data/structured/cnn_dailymail_data.csv') #path to CNN daily mail dataset
df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [4]:
# Create a Cognitive Search Index client
service_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")   
key = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")
credential = AzureKeyCredential(key)

index_name = os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME")

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
index_client

### Definir Campos del Índice y Crear una Configuración Semántica

Una *configuración semántica* especifica cómo se utilizan los campos en la clasificación semántica. Da a los modelos subyacentes pistas sobre qué campos de índice son más importantes para la clasificación semántica, los subtítulos, los destacados y las respuestas.

Puedes agregar o actualizar una configuración semántica en cualquier momento sin reconstruir tu índice. Cuando emites una consulta, añadirás la configuración semántica (una por consulta) que especifica qué configuración semántica usar para la consulta.

Revisa las propiedades que necesitarás especificar. Una configuración semántica tiene un nombre y al menos una de cada una de las siguientes propiedades:

* Campo de título: Un campo de título debe ser una descripción concisa del documento, idealmente una cadena de menos de 25 palabras. Este campo podría ser el título del documento, el nombre del producto o el elemento en tu índice de búsqueda. Si no tienes un título en tu índice de búsqueda, deja este campo en blanco.
* Campos de contenido: Los campos de contenido deben contener texto en formato de lenguaje natural. Ejemplos comunes de contenido son el cuerpo de un documento, la descripción de un producto u otro texto en forma libre.
* Campos de palabras clave: Los campos de palabras clave deben ser una lista de palabras clave, como las etiquetas en un documento, o un término descriptivo, como la categoría de un artículo.

Solo puedes especificar un campo de título, pero puedes especificar tantos campos de contenido y palabras clave como desees. Para los campos de contenido y palabras clave, enumera los campos en orden de prioridad porque los campos de menor prioridad pueden ser truncados.

In [5]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="highlights", type=SearchFieldDataType.String,
                searchable=True, retrievable=True),
    SearchableField(name="article", type=SearchFieldDataType.String,
                filterable=True, searchable=True, retrievable=True),
]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        #title_field=SemanticField(field_name=""), # title field is not present in the dataset. We can use OpenAI to generate title
        #prioritized_keywords_fields=[SemanticField(field_name="")], # keywords are not present in the dataset. We can use OpenAI to generate keywords
        prioritized_content_fields=[SemanticField(field_name="article"), SemanticField(field_name="highlights")]
    )
)


# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 news-index-labp2 created


In [6]:
# Convert the dataframe to a list of dictionaries
documents = df.to_dict('records')
documents[0]

{'id': '92c514c913c0bdfe25341af9fd72b29db544099b',
 'article': "Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crow

In [7]:
len(documents)

11490

In [8]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded and Indexed {len(result)} documents") 

Uploaded and Indexed 11490 documents


## Sección 1: Aprovechar la Búsqueda Cognitiva para extraer artículos relevantes basados en la consulta

In [9]:
#Extracting relevant article based on the query. eg: Clinton Democratic Nomination
results = search_client.search(search_text="Clinton Democratic nomination", include_total_count=True)
document = next(results)['article']

In [10]:
document

'(CNN)Hillary Clinton is now officially a candidate for president -- and the never ending Clinton story rumbles on. She has been a part of all our lives now for approaching a quarter of a century. She started as the first lady that the right loved to hate, then the deceived wife, next a senator, then a candidate for president in one of the most dynamic primaries in history and finally, a secretary of state. The Republicans have their aristocratic Bushes, the Democrats have their Clintons. And if Hillary or Jeb were to win two presidential terms, then in the 44 years from 1981 to 2025, 28 will have had a Clinton or a Bush in the White House. The great American republic now looks about as democratic as "Game of Thrones." But even though Hillary Clinton has been around nearly my entire lifetime, The Economist may speak for many when it asks: "What does Hillary stand for?" There is a paradox she presents: She is by far the best-known presidential candidate across both parties and, for the 

In [11]:
#length of article extracted from Azure Cognitive search
len(document) 

5714

## Sección 2: Búsqueda de Zona de Documentos
### Zona de Documentos: API de Embeddings de Azure OpenAI
Ahora que nos hemos enfocado en un solo documento de nuestra base de conocimientos usando Azure AI Search, podemos profundizar en dicho documento único para refinar nuestra consulta inicial a una sección más específica o "zona" del artículo.

Para hacer esto, utilizaremos la API de Embeddings de Azure OpenAI.

### **Descripción general de los Embeddings**
Un embedding es un formato especial de representación de datos que puede ser fácilmente utilizado por modelos y algoritmos de machine learning. El embedding es una representación densa en información del significado semántico de un fragmento de texto. Cada embedding es un vector de números de punto flotante, de modo que la distancia entre dos embeddings en el espacio vectorial está correlacionada con la similitud semántica entre dos entradas en el formato original. Por ejemplo, si dos textos son similares, entonces sus representaciones vectoriales también deberían ser similares.

Diferentes modelos de embedding de Azure OpenAI están específicamente creados para ser buenos en una tarea particular. Los embeddings de similitud son buenos para capturar la similitud semántica entre dos o más fragmentos de texto. Los embeddings de búsqueda de texto ayudan a medir qué documentos largos son relevantes para una consulta corta. Los embeddings de búsqueda de código son útiles para vectorizar fragmentos de código y consultas de búsqueda en lenguaje natural.

Los embeddings facilitan el machine learning en grandes entradas que representan palabras al capturar las similitudes semánticas en un espacio vectorial. Por lo tanto, podemos usar embeddings para determinar si dos fragmentos de texto están semánticamente relacionados o son similares, y de manera inherente proporcionar una puntuación para evaluar la similitud.

### **Similitud de Coseno**
Un enfoque utilizado anteriormente para localizar documentos similares se basaba en contar el número máximo de palabras comunes entre documentos. Esto es defectuoso ya que, a medida que aumenta el tamaño del documento, aumenta la superposición de palabras comunes incluso si los temas difieren. Por lo tanto, la similitud del coseno es un mejor enfoque.

Matemáticamente, la similitud del coseno mide el coseno del ángulo entre dos vectores proyectados en un espacio multidimensional. Esto es beneficioso porque si dos documentos están muy separados por la distancia euclidiana debido al tamaño, aún podrían tener un ángulo más pequeño entre ellos y, por lo tanto, una mayor similitud del coseno.

Los embeddings de Azure OpenAI se basan en la similitud del coseno para calcular la similitud entre documentos y una consulta.

## Configuración del servicio de Azure OpenAI y uso de modelos implementados

### **Chunking**

Comencemos con la segmentación (chunking). ¿Por qué es importante chunking al trabajar con LLMs?

Chunking ayuda a superar los desafíos asociados con el procesamiento de secuencias largas y garantiza un rendimiento óptimo al trabajar con LLMs.

**Mitigación de Limitaciones de Tokens:** Los LLMs tienen un límite máximo de tokens para cada secuencia de entrada. Si un documento o entrada excede este límite, necesita dividirse en fragmentos que se ajusten a las restricciones de tokens. Chunking permite que el LLM maneje documentos largos o entradas dividiéndolos en múltiples fragmentos que caen dentro del límite de tokens. Esto asegura que el modelo pueda procesar efectivamente todo el contenido mientras se adhiere a las restricciones de tokens.

**Eficiencia de Memoria y Computacional:** Los LLMs son computacionalmente costosos y requieren recursos de memoria sustanciales para procesar secuencias largas de texto. Chunking implica descomponer documentos largos o entradas en fragmentos más pequeños y manejables, permitiendo que el LLM los procese eficientemente dentro de sus limitaciones de memoria. Al dividir la entrada en partes más pequeñas, chunking ayuda a evitar errores de memoria o degradación del rendimiento que pueden ocurrir al procesar secuencias largas.

**Coherencia Contextual:** Chunking ayuda a mantener la coherencia contextual en las salidas generadas. En lugar de tratar toda la entrada como una sola secuencia, dividirla en fragmentos más pequeños permite que el modelo capture el contexto local de manera más efectiva. Esto mejora la comprensión del modelo de las relaciones y dependencias dentro del texto, lo que lleva a respuestas generadas más coherentes y significativas.

**Paralelismo Mejorado:** Chunking permite el procesamiento paralelo, lo cual es esencial para optimizar el rendimiento de los LLMs. Al dividir la entrada en fragmentos, se pueden procesar múltiples fragmentos simultáneamente, aprovechando las capacidades de computación paralela. Esto conduce a tiempos de inferencia más rápidos y mejora la eficiencia general al trabajar con LLMs.

Utilizaremos un splitter básico para este notebook. Sin embargo, es importante tener en cuenta que existen splitters más avanzados disponibles, que pueden adaptarse mejor a tu caso de uso específico.

In [12]:
#Defining helper functions
#Splits text after sentences ending in a period. Combines n sentences per chunk.
def splitter(n, s):
    pieces = s.split(". ")
    list_out = [" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n)]
    return list_out

# Perform light data cleaning (removing redudant whitespace and cleaning up punctuation)
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()    
    return s

In [13]:
document_chunks = splitter(10, normalize_text(document)) #splitting extracted document into chunks of 10 sentences
document_chunks

['(CNN)Hillary Clinton is now officially a candidate for president -- and the never ending Clinton story rumbles on She has been a part of all our lives now for approaching a quarter of a century She started as the first lady that the right loved to hate, then the deceived wife, next a senator, then a candidate for president in one of the most dynamic primaries in history and finally, a secretary of state The Republicans have their aristocratic Bushes, the Democrats have their Clintons And if Hillary or Jeb were to win two presidential terms, then in the 44 years from 1981 to 2025, 28 will have had a Clinton or a Bush in the White House The great American republic now looks about as democratic as "Game of Thrones." But even though Hillary Clinton has been around nearly my entire lifetime, The Economist may speak for many when it asks: "What does Hillary stand for?" There is a paradox she presents: She is by far the best-known presidential candidate across both parties and, for the mome

In [14]:
# Handling Rate Limits

from openai.error import RateLimitError
from time import sleep


def get_embedding(text: str, engine: str = "text-embedding-ada-002"):
    count=0
    while True:
        try:
            embedding = openai.Embedding().create(input=[text], engine=engine)["data"][0]["embedding"]
            break;
        except RateLimitError:
            count+=1
            #print(f'RateLimitError Count: {count}')
            sleep(2)            
    return np.array(embedding).astype(np.float32)

def get_completion(prompt, model="gpt-35-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [15]:
embed_df = pd.DataFrame(document_chunks, columns = ["chunks"]) #datframe with document chunks

#Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
embed_df['embeddings'] = embed_df["chunks"].apply(lambda x : get_embedding(x, engine = embedding_model))

embed_df 


,chunks,embeddings
0,(CNN)Hillary Clinton is now officially a candi...,"[-0.02244035, -0.010071911, -0.0031363997, -0...."
1,"The riots in Ferguson, Missouri, were a painfu...","[-0.005809085, -0.029980645, -0.015955709, -0...."
2,"In Iowa last week, both Jim Webb and Martin O'...","[-0.014524365, -0.0053608236, 0.026020017, -0...."
3,"But, since then, Democratic power has been whi...","[-0.021648454, -0.0040004035, 0.011662534, -0...."


In [16]:
# search through the document for a text segment most similar to the query
# display top two most similar chunks based on cosine similarity
def search_docs(df, user_query, top_n=3):
    embedding = get_embedding(
        user_query,
        engine=embedding_model,
    )
    df["similarities"] = df['embeddings'].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .reset_index(drop=True)
        .head(top_n)
    )
    return res

In [17]:
document_specific_query = "trouble so far in clinton campaign" 
res = search_docs(embed_df, document_specific_query, top_n=2) #finding top 2 results based on similarity 
res

,chunks,embeddings,similarities
0,"In Iowa last week, both Jim Webb and Martin O'...","[-0.014524365, -0.0053608236, 0.026020017, -0....",0.848953
1,"The riots in Ferguson, Missouri, were a painfu...","[-0.005809085, -0.029980645, -0.015955709, -0....",0.843543


## Sección 3: Resumen de Texto

Esta sección trata del flujo de principio a fin del uso de los modelos GPT-3 y ChatGPT para tareas de resumen. El modelo utilizado por el servicio Azure OpenAI es una llamada de completado generativo que utiliza instrucciones en lenguaje natural para identificar la tarea solicitada y la habilidad requerida, también conocido como Prompt Engineering. Usando este enfoque, la primera parte del prompt incluye instrucciones en lenguaje natural y/o ejemplos de la tarea específica deseada. Luego, el modelo completa la tarea prediciendo el próximo texto más probable. Esta técnica se conoce como aprendizaje "en contexto".

Hay tres enfoques principales para el aprendizaje en contexto: Zero-shot, Few-shot y Fine-tuning. Estos enfoques varían según la cantidad de datos específicos de la tarea que se proporcionan al modelo:

**Zero-shot (cero disparos):** En este caso, no se proporcionan ejemplos al modelo y solo se proporciona la solicitud de la tarea.

**Few-shot (pocos disparos):** En este caso, un usuario incluye varios ejemplos en el prompt de llamada que demuestran el formato y el contenido de la respuesta esperada.

**Fine-Tuning (ajuste preciso):** Fine Tuning te permite adaptar los modelos a tus conjuntos de datos personales. Este paso de personalización te permitirá obtener más del servicio proporcionando:
-   Con muchos datos (al menos 500 y más), se utilizan técnicas de optimización tradicionales con Back Propagation para reajustar los pesos del modelo - esto permite obtener resultados de mayor calidad que las técnicas simples de zero-shot o few-shot.
-   Un modelo personalizado mejora el enfoque de few-shot learning (aprendizaje de pocos disparos) al entrenar los pesos del modelo en tus prompts y estructuras específicas. Esto te permite lograr mejores resultados en una mayor cantidad de tareas sin necesidad de proporcionar ejemplos en el prompt. El resultado es menos texto enviado y menos tokens.

In [18]:
'''Designing a prompt that will show and tell GPT-3 how to proceed. 
+ Providing an instruction to summarize the text about the general topic (prefix)
+ Providing quality data for the chunks to summarize and specifically mentioning they are the text provided (context + context primer)
+ Providing a space for GPT-3 to fill in the summary to follow the format (suffix)
'''

# result_1 corresponding to the top chunk from Section 2. result_2 corresponding to the second to top chunk from section 2. 
# change index for desired chunk
result_1 = res.chunks[0]
result_2 = res.chunks[1]
prompt_i = 'Summarize the content about the Clinton campaign given the text provided.\n\nText:\n'+" ".join([normalize_text(result_1)])+ '\n\nText:\n'+ " ".join([normalize_text(result_2)])+'\n\nSummary:\n'
print(prompt_i)

Summarize the content about the Clinton campaign given the text provided.

Text:
In Iowa last week, both Jim Webb and Martin O'Malley attacked Wall Street, Webb adding that he had also opposed the Iraq War Both men questioned the wisdom of Clintonian triangulation -- the idea that the White House can be won, and the country successfully governed, by always seeking the middle ground Both men would be wise to focus on Iowa; to contrast a populist, folksy campaign with the distant, over-managed style of Clinton And both would do well to tap into a feeling that it would be unhealthy, undemocratic and plain dull to let Hillary coast to the nomination without a proper challenge Nevertheless, there is a strange contradiction between the constant assertions that Democrats want a race and the polling evidence that Clinton would beat anyone who tried to take her on Why do liberals demand a conversation about policy if the only answer they can still come up with is Hillary? The explanation is tha

In [19]:
get_completion(prompt_i, model=chat_model) # default temperature is set to 0

"The Clinton campaign is facing criticism from Democratic rivals who are questioning her credentials as a fighter against inequality and her foreign policy record. Her friends in corporate America and the Clinton Foundation's fundraising efforts among foreign interests are also being scrutinized. Her personal ethics, including the use of a private email account, are also on the agenda. Despite calls for a proper challenge, polling evidence suggests that Clinton would beat anyone who tried to take her on, highlighting the intellectual impoverishment of the Democratic Party."

In [20]:
# bumping up the temperature to 0.5 to increase the randomness of the model's output
get_completion(prompt_i, model=chat_model, temperature=0.5)

"The Clinton campaign is facing challenges from Democratic rivals who are questioning her credentials as a fighter against inequality and her foreign policy record. Her ties to corporate interests and the Clinton Foundation's fundraising efforts among foreign interests are also being scrutinized. The Democratic Party is intellectually impoverished and needs a proper challenge to avoid letting Clinton coast to the nomination. Despite demands for a conversation about policy, polling evidence suggests that Clinton would beat anyone who tried to take her on."